# Part 1

In [107]:
import re



data = []
grid_min_x = grid_min_y = grid_max_x = grid_max_y = 0

with open('data/day-15.data') as f:
    positions_pattern = re.compile(r'Sensor\sat\s(.*):\sclosest\sbeacon\sis\sat\s(.*)')
    coordinate_pattern = re.compile(r'x=(.*),\sy=(.*)')

    for line in f.readlines():
        positions_matches = positions_pattern.match(line.strip())
        sensor_pos_matches = coordinate_pattern.match(positions_matches.group(1))
        beacon_pos_matches = coordinate_pattern.match(positions_matches.group(2))

        sensor_x = int(sensor_pos_matches.group(1))
        sensor_y = int(sensor_pos_matches.group(2))
        beacon_x = int(beacon_pos_matches.group(1))
        beacon_y = int(beacon_pos_matches.group(2))

        if sensor_x > grid_max_x: grid_max_x = sensor_x
        if beacon_x > grid_max_x: grid_max_x = beacon_x
        if sensor_y > grid_max_y: grid_max_y = sensor_y
        if beacon_y > grid_max_y: grid_max_y = beacon_y

        if sensor_x < grid_min_x: grid_min_x = sensor_x
        if beacon_x < grid_min_x: grid_min_x = beacon_x
        if sensor_y < grid_min_y: grid_min_y = sensor_y
        if beacon_y < grid_min_y: grid_min_y = beacon_y

        data.append([(sensor_x, sensor_y), (beacon_x, beacon_y), abs(sensor_x - beacon_x) + abs(sensor_y - beacon_y)])



print(grid_min_x, grid_max_y)
print(grid_min_y, grid_max_x)

len(data)

0 4188971
-158441 4171278


32

In [86]:
def get_manhatten_dist(source, dest):
    return abs(dest[0] - source[0]) + abs(dest[1] - source[1])

In [88]:
# TARGET_ROW = 2_000_000
TARGET_ROW = 20

covered_target_cells = set()
beacons_on_target = set()

for datum in data:
    sensor_pos = datum[0]
    beacon_pos = (datum[1][0] + 1, datum[1][1])
    sensor_range = datum[2]

    min_x = sensor_pos[0] - sensor_range
    max_x = sensor_pos[0] + sensor_range
    min_y = sensor_pos[1] - sensor_range
    max_y = sensor_pos[1] + sensor_range

    if min_y <= TARGET_ROW <= max_y:
        for x in range(min_x, max_x):
            if get_manhatten_dist((x, TARGET_ROW), sensor_pos) <= sensor_range:
                covered_target_cells.add((x, TARGET_ROW))

            if (x, TARGET_ROW) == beacon_pos: beacons_on_target.add((x, TARGET_ROW))



len(covered_target_cells) - len(beacons_on_target)

26

# Part 2

In [89]:
def is_point_covered(point, beacons_positions):
    for sensor_pos, _, sensor_range in data:
        if get_manhatten_dist(point, sensor_pos) <= sensor_range:
            return True

    return False

In [114]:
import datetime
from tqdm import tqdm



MAX_RANGE = 4_000_000
# MAX_RANGE = 20

beacon_positions = set()
done = False

for _, beacon_pos, _ in data:
    beacon_positions.add(beacon_pos)

for sensor_pos, _, sensor_range in tqdm(data):
    sensor_x, sensor_y = sensor_pos

    if done: break

    # bottom-right quadrant
    for x in range(sensor_range + 2):
        y = sensor_range - (x - 1)
        x += sensor_x
        y += sensor_y

        if 0 <= x + sensor_x <= MAX_RANGE and 0 <= y + sensor_y <= MAX_RANGE and not is_point_covered((x, y), beacon_positions):
            print(x*4_000_000 + y)
            done = True
            break

    # bottom-left quadrant
    for y in range(sensor_range + 2):
        if done: break

        x = (y - 1) - sensor_range
        x += sensor_x
        y += sensor_y

        if 0 <= x <= MAX_RANGE and 0 <= y <= MAX_RANGE and not is_point_covered((x, y), beacon_positions):
            print(x*4_000_000 + y)
            done = True
            break

    # top-left quadrant
    for x in range(-1 * (sensor_range + 2), 1):
        if done: break

        y = -1 * sensor_range - (x - 1)
        x += sensor_x
        y += sensor_y

        if 0 <= x <= MAX_RANGE and 0 <= y <= MAX_RANGE and not is_point_covered((x, y), beacon_positions):
            print(x*4_000_000 + y)
            done = True
            break

    # top-right quadrant
    for y in range(-1 * (sensor_range -2), 1):
        if done: break

        x = sensor_range + (y + 1)
        x += sensor_x
        y += sensor_y

        if 0 <= x <= MAX_RANGE and 0 <= y <= MAX_RANGE and not is_point_covered((x, y), beacon_positions):
            print(x*4_000_000 + y)
            done = True
            break

 69%|██████▉   | 22/32 [03:34<01:37,  9.73s/it]

11840879211051
